### Experiment: Generalization


**Question**: How well does each of the models (CNN, GRU, Transformer) generalize to other datasets?

**Hypothesis**: The models differ in generalizability, ranging from none at all to above-chance performance

**Result**:

How to perform this?
- Take model trained on 100 Hz SAT1
- Test on test-set
- Test on entire set for SAT2 and AR:
`train_data, val_data, test_data = split_data_on_participants(
    dataset, 100, norm_min1_to_1
)`

In [6]:
%load_ext autoreload
%autoreload 2
import netCDF4
import xarray as xr
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model
from hmpai.pytorch.generators import SAT1Dataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT1_STAGES_SPEED
from hmpai.visualization import plot_confusion_matrix
from hmpai.normalization import *
from torchinfo import summary
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
# from braindecode.models.eegconformer import EEGConformer
from mne.io import read_info
import gc
import json

import os
from torch import nn
DATA_PATH = Path(os.getenv("DATA_PATH"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
set_global_seed(42)

data_path = DATA_PATH / "sat2/split_stage_data_100hz.nc"
# data_path = Path("../data/sat2/split_stage_data_100hz.nc")

dataset = xr.load_dataset(data_path)

In [9]:
# Split into acc and speed
# Use same participants for speed and acc
dataset_acc = dataset.where(dataset.event_name.str.contains("accuracy"), drop=True)
dataset_sp = dataset.where(dataset.event_name.str.contains("speed"), drop=True)
del dataset

In [12]:
def test_generalization(
    model_fn, model_kwargs, data, additional_test_data, additional_train_kwargs=None
):
    print(f"Testing model: {model_fn.__name__}")
    train_kwargs = {
        "logs_path": Path("../logs/exp_generalization_condition/"),
        "additional_info": {
            "model_fn": model_fn.__name__,
            "model_kwargs": str(model_kwargs),
        },
        "additional_name": f"model_fn-{model_fn.__name__}",
        "labels": SAT1_STAGES_ACCURACY,
    }
    if additional_train_kwargs is not None:
        train_kwargs.update(additional_train_kwargs)
    result = k_fold_cross_validate(
        model_fn,
        model_kwargs,
        data,
        k=len(data.participant),
        normalization_fn=norm_min1_to_1,
        train_kwargs=train_kwargs,
        additional_test_data=additional_test_data,
    )
    with open(
        train_kwargs["logs_path"] / f"results_{model_fn.__name__}.json", "w"
    ) as f:
        json.dump(result, f, indent=4)
    print_results(result)

#### CNN

In [ ]:
test_generalization(
    SAT1Base,
    {"n_classes": len(dataset_acc.labels)},
    dataset_acc,
    additional_test_data=[dataset_sp],
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

#### GRU

In [13]:
test_generalization(
    SAT1GRU,
    {
        "n_channels": len(dataset_acc.channels),
        "n_samples": len(dataset_acc.samples),
        "n_classes": len(dataset_acc.labels),
    },
    dataset_acc,
    additional_test_data=[dataset_sp],
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

Testing model: SAT1GRU
Fold 1: test fold: ['S1']


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/657 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       0.97      0.90      0.93       952
           1       0.65      0.88      0.75      1013
           2       0.58      0.72      0.64      1013
           3       0.74      0.30      0.43      1013
           4       0.89      0.95      0.92      1013

    accuracy                           0.75      5004
   macro avg       0.76      0.75      0.73      5004
weighted avg       0.76      0.75      0.73      5004



  0%|          | 0/657 [00:00<?, ? batch/s]

...

  0%|          | 0/662 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       0.97      0.84      0.90       797
           1       0.70      0.97      0.81       905
           2       0.99      0.87      0.93       905
           3       0.93      0.77      0.84       905
           4       0.89      0.90      0.90       905

    accuracy                           0.87      4417
   macro avg       0.90      0.87      0.88      4417
weighted avg       0.89      0.87      0.87      4417



  0%|          | 0/662 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       0.97      0.84      0.90       797
           1       0.51      0.97      0.67       905
           2       0.99      0.67      0.80       905
           3       0.94      0.45      0.60       905
           4       0.87      0.93      0.90       905

    accuracy                           0.77      4417
   macro avg       0.86      0.77      0.78      4417
weighted avg       0.86      0.77      0.77      4417

Fold 18: Accuracy: 0.9067240208286167
Fold 18: F1-Score: 0.9065477893957595
Test set 0
Accuracies
[0.8936850519584333, 0.9188675660269808, 0.9122304903644343, 0.8884998157021747, 0.903838986209467, 0.9165092557612391, 0.9112426035502958, 0.879415347137637, 0.8755413293165129, 0.8762798634812287, 0.8576728499156829, 0.9363790824995281, 0.9344750558451228, 0.9371428571428572, 0.9081345106143153, 0.9118951612903226, 0.9263002737418404, 0.9067240208286167]
F1-Scores
[0.8934026720096183, 0.9183081369815268, 0.9

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Transformer

In [8]:
test_generalization(
    TransformerModel,
    {
        "n_features": len(dataset_acc.channels),
        "n_heads": 10,
        "ff_dim": 512,
        "n_layers": 6,
        "n_samples": len(dataset_acc.samples),
        "n_classes": len(dataset_acc.labels),
    },
    dataset_acc,
    additional_test_data=[dataset_sp],
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

Testing model: TransformerModel
Fold 1: test fold: ['S1']


/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/657 [00:00<?, ? batch/s]

  0%|          | 0/657 [00:00<?, ? batch/s]

  0%|          | 0/657 [00:00<?, ? batch/s]

...

  0%|          | 0/662 [00:00<?, ? batch/s]

  0%|          | 0/662 [00:00<?, ? batch/s]

  0%|          | 0/662 [00:00<?, ? batch/s]

Fold 18: Accuracy: 0.8632556033506905
Fold 18: F1-Score: 0.8638125038123426
Test set 0
Accuracies
[0.8677058353317346, 0.9002470074102223, 0.8784583094829231, 0.8875783265757464, 0.8317182258665673, 0.8675859463543635, 0.8593688362919132, 0.8223710921640276, 0.8514404067030691, 0.6819539249146758, 0.8263069139966274, 0.8763451010005664, 0.9180938198064036, 0.9138285714285714, 0.9177155739244787, 0.8602822580645161, 0.8949252474205096, 0.8632556033506905]
F1-Scores
[0.8689372900374132, 0.9001277760402411, 0.8781092890157616, 0.8887251604606015, 0.8241451596932088, 0.8691268921871007, 0.8569738175833346, 0.8223208940280223, 0.8516950330657533, 0.6217590937764896, 0.8280214236628518, 0.8727961606486361, 0.9188519055576905, 0.9140924833061861, 0.9176603329151355, 0.8578540636618808, 0.8957739891746572, 0.8638125038123426]
Average Accuracy: 0.8621767222270893, std: 0.05216370114336046
Average F1-Score: 0.8583768482570726, std: 0.0643863180399913
Test set 1
Accuracies
[0.8906370905339179, 0.

/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu